In [1]:
import os

def find_env_file(start_dir="."):
    for root, dirs, files in os.walk(start_dir):
        if ".env" in files:
            return os.path.join(root, ".env")
    return None

# Example usage
env_file_path = find_env_file()
if env_file_path:
    print(f".env file found at: {env_file_path}")
else:
    print(".env file not found")

.env file found at: ./.env


In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Model and API Key

In [3]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Documents

In [9]:
pip install pypdf

  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/04/fc/6f52588ac1cb4400a7804ef88d0d4e00cfe57a7ac6793ec3b00de5a8758b/pypdf-5.1.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 527.1 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Additional imports for loading PDF documents and QA chain.
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [17]:
# Create the PDF loader and load the document.
pdf_loader = PyPDFLoader("../Resources/Resume.pdf")
documents = pdf_loader.load()

# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=OPENAI_MODEL, temperature=0.0)

# Create the QA chain using the LLM.
chain = load_qa_chain(llm)

# Define a query as a string.
query = "Could I write to Jane doe is spanish and expect her to understand?"

# Pass the documents and the query to the chain, and print the result.
result = chain.invoke({"input_documents" : documents, "question" : query})
print(result["output_text"])

Yes, Jane Doe mentions in her resume that she has a conversational level of Spanish, so it's possible that she may understand your message if you write to her in Spanish. However, it's always a good idea to confirm with her directly if she is comfortable communicating in Spanish.


# Built-in Integrations

In [11]:
# Additional imports for loading Wikipedia content and QA chain.
from langchain_community.document_loaders import WikipediaLoader
from langchain.chains.question_answering import load_qa_chain


In [19]:
# Initialize the model.
llm = ChatOpenAI(#openai_api_key = OPENAI_API_KEY, 
                 model = OPENAI_MODEL,
                 temperature = 0.0)

# Define the wikipedia topic as a string.
wiki_topic = "The Beatles"

# Load the wikipedia results as documents, using a max of 2.
documents = WikipediaLoader(query=wiki_topic,
                            load_max_docs=2,
                            load_all_available_meta=True).load()

# Create the QA chain using the LLM.
chain = load_qa_chain(llm)

# Define a query as a string.
query = 'How many albums have The Beatles released?'

# Pass the documents and the query to the chain, and print the result.
result = chain.invoke({"input_documents" : documents,
                       "question" : query})
print(result["output_text"])

The Beatles released a total of 12 studio albums during their career.


# External APIs

In [15]:
# Additional imports for API chain.
from langchain.chains import APIChain

In [18]:
# Text description of API spec.
spec = """URL STRUCTURE
        Just hit http://numbersapi.com/number/type to get a plain text response, where
        type is one of trivia, math, date, or year. Defaults to trivia if omitted.
        number is an integer, or the keyword random, for which we will try to return
        a random available fact, or a day of year in the form month/day 
        (eg. 2/29, 1/09, 04/1), if type is date ranges of numbers"""

# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, 
                 model=OPENAI_MODEL, 
                 temperature=0)

# Create the API chain from the spec, using the LLM.
chain = APIChain.from_llm_and_api_docs(
    llm=llm,
    api_docs = spec,
    verbose = True,
    limit_to_domains=['http://numbersapi.com'],
) 

# Define a query as a dictionary.
query = {"question" : "What is a fun fact about the number 2?"}

# Run the chain using the query, and print the response.
chain.invoke(query)



> Entering new APIChain chain...
http://numbersapi.com/2/trivia
2 is the number of polynucleotide strands in a DNA double helix.

> Finished chain.


{'question': 'What is a fun fact about the number 2?',
 'output': 'A fun fact about the number 2 is that it is the number of polynucleotide strands in a DNA double helix.'}

In [20]:
# Additional imports for API chain and processing json.
from langchain.chains import APIChain
import json
import requests

In [21]:
# Store the API key in a variable.
api_key = os.getenv("NYT_API_KEY")

# Load the spec from json file and store the API key.
url = "https://raw.githubusercontent.com/nytimes/public_api_specs/master/article_search/article_search_v2.json"

spec = requests.get(url).json()
spec["api_key"] = api_key

# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)

# Create the API chain from the spec, using the LLM.
chain = APIChain.from_llm_and_api_docs(
    llm,
    api_docs=json.dumps(spec),
    verbose=False,  # Changing to True will expose user's NYT API key.
    limit_to_domains=["https://api.nytimes.com/"],
)

# Define a query as a string.
query = {"question": 'What are the headlines of three articles about Tracy Chapman since 2020?'}

# Run the chain using the query, and print the response.
result = chain.invoke(query)
print(result["output"])


The headlines of three articles about Tracy Chapman since 2020 are:
1. "Covering the Rise of Tracy Chapman"
2. "Where Has Tracy Chapman Been?"
3. "GWS-tracy-chapman-fast-car-grammys"
